In [10]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
from scipy import signal
from pywt import wavedec

In [2]:
pickles_address = '/content/drive/MyDrive/Brain-Computer Interfaces/Epilepsy Dataset/Pickles/'
A_address = '/content/drive/MyDrive/Brain-Computer Interfaces/Epilepsy Dataset/Z/'
B_address = '/content/drive/MyDrive/Brain-Computer Interfaces/Epilepsy Dataset/O/'
C_address = '/content/drive/MyDrive/Brain-Computer Interfaces/Epilepsy Dataset/N/'
D_address = '/content/drive/MyDrive/Brain-Computer Interfaces/Epilepsy Dataset/F/'
E_address = '/content/drive/MyDrive/Brain-Computer Interfaces/Epilepsy Dataset/S/'

In [5]:
with open(pickles_address + 'As.pickle', 'rb') as f:
    As = pickle.load(f)
with open(pickles_address + 'Bs.pickle', 'rb') as f:
    Bs = pickle.load(f)
with open(pickles_address + 'Cs.pickle', 'rb') as f:
    Cs = pickle.load(f)
with open(pickles_address + 'Ds.pickle', 'rb') as f:
    Ds = pickle.load(f)
with open(pickles_address + 'Es.pickle', 'rb') as f:
    Es = pickle.load(f)
print('As', As.shape)
print('Bs', Bs.shape)
print('Cs', Cs.shape)
print('Ds', Ds.shape)
print('Es', Es.shape)

As (4097, 100)
Bs (4097, 100)
Cs (4097, 100)
Ds (4097, 100)
Es (4097, 100)


In [4]:
def butter_bandstop_filter(data, lowcut, highcut, fs, order):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = signal.butter(order, [low, high], btype='bandstop')
    y = signal.filtfilt(b, a, data)
    return y

def butter_bandpass_filter(data, lowcut, highcut, fs, order):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = signal.butter(order, [low, high], btype='bandpass')
    y = signal.filtfilt(b, a, data)
    return y

In [6]:
from statistics import mean
from scipy.stats import skew, kurtosis, entropy

n = 2000
fs = 173.61
fd = 120

In [7]:
As_features = []

for i in range(As.shape[1]):
    trial = butter_bandstop_filter(As[:, i], 49, 51, fs, 3)
    m = mean(trial)
    v = np.var(trial)
    s = skew(trial)
    k = kurtosis(trial)
    e = entropy(trial)
    p = mean(trial ** 2)
    r = np.sqrt(np.mean(trial**2))
    As_features.append([m, v, s, k, e, p, r])

As_features = np.array(As_features).T
print(As_features.shape)

(7, 100)


In [8]:
list(As_features[:, 0])

[-52.44446150206143,
 2381.2296567113876,
 -0.09267826028757002,
 0.6267915072206152,
 -inf,
 5131.651198952592,
 71.63554424273325]

In [9]:
def feature_extractor(inp):
    inp_features = []
    for i in range(inp.shape[1]):
        trial = butter_bandstop_filter(inp[:, i], 49, 51, fs, 3)
        m = mean(trial)
        v = np.var(trial)
        s = skew(trial)
        k = kurtosis(trial)
        e = entropy(trial)
        p = mean(trial ** 2)
        r = np.sqrt(np.mean(trial**2))
        inp_features.append([m, v, s, k, e, p, r])

    return np.array(inp_features).T

As_features = feature_extractor(As)
Bs_features = feature_extractor(Bs)
Cs_features = feature_extractor(Cs)
Ds_features = feature_extractor(Ds)
Es_features = feature_extractor(Es)

As_features.shape, Bs_features.shape, Cs_features.shape, Ds_features.shape, Es_features.shape

((7, 100), (7, 100), (7, 100), (7, 100), (7, 100))

In [12]:
def rhyme_based_feature_extractor(inp):
    inp_features = []
    for i in range(inp.shape[1]):
        level = 4
        wname = 'db6'
        coefficients = wavedec(inp[:, i], level=level, wavelet=wname)
        L = [len(each) for each in coefficients]
        trial_delta, trial_theta, trial_alpha, trial_beta, trial_gamma = coefficients
        trial_rhymes = [trial_delta, trial_theta, trial_alpha, trial_beta, trial_gamma]
        rhymes_features = []
        for each_rhyme in trial_rhymes:
            m = mean(each_rhyme)
            v = np.var(each_rhyme)
            s = skew(each_rhyme)
            k = kurtosis(each_rhyme)
            e = entropy(each_rhyme)
            p = mean(each_rhyme ** 2)
            r = np.sqrt(np.mean(trial**2))
            rhymes_features.append([m, v, s, k, e, p, r])
        inp_features.append(np.array(rhymes_features).reshape(35, 1))
    return np.array(inp_features).T[0]


As_rhyme_features = rhyme_based_feature_extractor(As)
Bs_rhyme_features = rhyme_based_feature_extractor(Bs)
Cs_rhyme_features = rhyme_based_feature_extractor(Cs)
Ds_rhyme_features = rhyme_based_feature_extractor(Ds)
Es_rhyme_features = rhyme_based_feature_extractor(Es)

As_rhyme_features.shape, Bs_rhyme_features.shape, Cs_rhyme_features.shape, Ds_rhyme_features.shape, Es_rhyme_features.shape

((35, 100), (35, 100), (35, 100), (35, 100), (35, 100))

In [13]:
with open(pickles_address + 'As_features_TF.pickle', 'wb') as f:
    pickle.dump(As_features, f)
with open(pickles_address + 'Bs_features_TF.pickle', 'wb') as f:
    pickle.dump(Bs_features, f)
with open(pickles_address + 'Cs_features_TF.pickle', 'wb') as f:
    pickle.dump(Cs_features, f)
with open(pickles_address + 'Ds_features_TF.pickle', 'wb') as f:
    pickle.dump(Ds_features, f)
with open(pickles_address + 'Es_features_TF.pickle', 'wb') as f:
    pickle.dump(Es_features, f)
with open(pickles_address + 'As_rhyme_features_TF.pickle', 'wb') as f:
    pickle.dump(As_rhyme_features, f)
with open(pickles_address + 'Bs_rhyme_features_TF.pickle', 'wb') as f:
    pickle.dump(Bs_rhyme_features, f)
with open(pickles_address + 'Cs_rhyme_features_TF.pickle', 'wb') as f:
    pickle.dump(Cs_rhyme_features, f)
with open(pickles_address + 'Ds_rhyme_features_TF.pickle', 'wb') as f:
    pickle.dump(Ds_rhyme_features, f)
with open(pickles_address + 'Es_rhyme_features_TF.pickle', 'wb') as f:
    pickle.dump(Es_rhyme_features, f)